In [1]:
import os
import pandas as pd
import numpy as np
import itertools as it

In [11]:
sub = 'pilot-54'
ses = 'ses-01'
battery = 'fbirn'
task = 'checkerboard-react'

main_dir = '/home/sshankar/ibc/analysis_pipeline/ibc_main/neurospin_data/info/'
# main_dir = '/home/sshankar/Documents/Data/'
datadir = os.path.join(main_dir, sub)
csv_file = 'extracted_checkerboard-react_sub-54_ses-02_2112021-1215.csv'
tsv_file = 'checkerboard-react_pilot-54_ses_02.tsv'

In [12]:
file_df = pd.read_csv(os.path.join(datadir, csv_file))

In [13]:
# In the csv file extracted from the E-Prime txt file, 
# run onset is the 'GetReady.OffsetTime' column of last row, 
# and all other columns of this row are empty.
run_start = (file_df.loc[~np.isnan(file_df['sscanstart1.OffsetTime'])])['sscanstart1.OffsetTime']
run_start = run_start.values[0]

In [14]:
run_start

50907.0

In [15]:
# Get trial_types names from Procedure
# trial_types = (file_df.loc['Block' in file_df['Procedure']])['Procedure']
types = file_df['Procedure']
trials = types.str.contains("Block", regex=False)
trial_types = types[trials==True] 
trial_types[trial_types=='fixationBlock'] = "fixation"
trial_types[trial_types=='checkerBlock'] = "checkerboard"

In [16]:
# Get trial_types names from Procedure. This is a block design experiment 
# but there are trials of fix and checkerboard within each block. Blocks are 
# separated with Procedure names of "fixationBlock" and "checkerBlock".
types = file_df['Procedure']
trials = types.str.contains("Block", regex=False)
trial_types = types[trials==True] 
trial_types[trial_types=='fixationBlock'] = "fixation"
trial_types[trial_types=='checkerBlock'] = "checkerboard"

# Onset times will be in the indices 0 and the one AFTER the trial_types index
# The run begins and ends with a fixation block
onsets = []

# First fixation block
onsets.append(file_df.loc[0]['fixation.OnsetTime'])
    
# Subsequent runs
for idx in trial_types.keys():
    if trial_types[idx] == "fixation":
        onsets.append(file_df.loc[idx+1]['Stimulus.OnsetTime'])
    else:
        onsets.append(file_df.loc[idx+1]['fixation.OnsetTime'])

# Final key is a NaN so remove it
onsets.pop()

# Calculate durations
durations = np.diff(onsets)
durations = np.append(durations, file_df.iloc[-3]['fixation.OnsetTime']-onsets[-1])

In [17]:
# Create a DataFrame to save to events file
df = pd.DataFrame({'onset':(onsets-run_start)/1000, 'duration':durations/1000, 'trial_type':trial_types})
df.to_csv(os.path.join(datadir, tsv_file), sep='\t', float_format='%0.3f', index=False)